In [1]:
import pandas as pd
from pathlib import Path

# 1. Read the file produced by `assign_polys.py`

In [85]:
# Path to data file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
data_path = Path("C:\\Users\\tbergamaschi\\Data\\RMP\\ToeDrainFlowPulse2018\\SpatialJoin\\2018-10-04_fts_20secMed_QAQC_ebs_110419_spatialJoin.csv")

df_data = pd.read_csv(data_path)
df_data.head()

,TIMESTAMP,Latitude_left,Longitude_left,WS_fDOM_QSU,WS_fCHLA_ug/L,SUNANO3uM,CStar_Tr,CStar_c,TSG_Temp,TSG_Cond,...,EXOTurbFNU,EXOfDOM_QSU,EXOfChla_ug/L,index_right,Location,Latitude_right,Longitude_right,Shape_Leng,Shape_Area,Poly_ID_01
0,2018-10-04 08:29:56,38.15562,-121.682660,20.605298,1.504000,14.345,0.729710,3.151085,19.46790,127.4,...,2.81,14.297678,NaN,4410.0,Sacramento River,38.156438,-121.685087,0.019328,0.000012,677.0
1,2018-10-04 08:29:57,38.15562,-121.682655,20.603970,1.505902,14.340,0.729709,3.151097,19.46780,127.4,...,2.81,14.316774,NaN,4410.0,Sacramento River,38.156438,-121.685087,0.019328,0.000012,677.0
2,2018-10-04 08:29:58,38.15562,-121.682650,20.604907,1.509847,14.345,0.729550,3.153269,19.46770,127.4,...,2.81,14.327609,NaN,4410.0,Sacramento River,38.156438,-121.685087,0.019328,0.000012,677.0
3,2018-10-04 08:29:59,38.15562,-121.682650,20.603970,1.513792,14.340,0.729392,3.155441,19.46760,127.4,...,2.81,14.338444,NaN,4410.0,Sacramento River,38.156438,-121.685087,0.019328,0.000012,677.0
4,2018-10-04 08:30:00,38.15562,-121.682650,20.602493,1.524879,14.340,0.729247,3.157426,19.46755,127.4,...,2.81,14.344508,NaN,4410.0,Sacramento River,38.156438,-121.685087,0.019328,0.000012,677.0


# 2. A little bit of clean up

It doesn't make sense to compute statistics for some columns in the dataset (like latitude and longitude), so lets identify the columns we don't want to include and drop them from the dataframe.

*Note: we could do this the other way - by identifying the columns we want to keep - but it is my hope that the columns we want to drop will be more or less consistant from dataset to dataset, more so than the constituent columns. So hopefully identifying the columns to drop will mean that little or no modification is needed from dataset to dataset. We will see...*

In [86]:
# First lets see what columns we have
df_data.columns.values

array(['TIMESTAMP', 'Latitude_left', 'Longitude_left', 'WS_fDOM_QSU',
       'WS_fCHLA_ug/L', 'SUNANO3uM', 'CStar_Tr', 'CStar_c', 'TSG_Temp',
       'TSG_Cond', 'TSG_Salinity', 'EXOTemp', 'EXOSpCond', 'EXOpH',
       'EXOpHmV', 'EXODOmgL', 'EXODOSAT', 'EXOTurbFNU', 'EXOfDOM_QSU',
       'EXOfChla_ug/L', 'index_right', 'Location', 'Latitude_right',
       'Longitude_right', 'Shape_Leng', 'Shape_Area', 'Poly_ID_01'],
      dtype=object)

In [87]:
# Now lets make a list of the ones we don't want to include
# I am just copy/pasting non-constituent columns from above into this list
cols_to_drop = [
    'TIMESTAMP', 'Latitude_left', 'Longitude_left', 'index_right',
       'Location', 'Latitude_right', 'Longitude_right', 'Shape_Leng',
       'Shape_Area',
    'Shape_Leng', 'Shape_Area'
]

In [88]:
df_data = df_data.drop(cols_to_drop, axis="columns")
df_data.columns.values

array(['WS_fDOM_QSU', 'WS_fCHLA_ug/L', 'SUNANO3uM', 'CStar_Tr', 'CStar_c',
       'TSG_Temp', 'TSG_Cond', 'TSG_Salinity', 'EXOTemp', 'EXOSpCond',
       'EXOpH', 'EXOpHmV', 'EXODOmgL', 'EXODOSAT', 'EXOTurbFNU',
       'EXOfDOM_QSU', 'EXOfChla_ug/L', 'Poly_ID_01'], dtype=object)

Cool now we just have columns that we are going to use in our statistics computation. Now we need to identify the column that contains the polygon id that was assigned by `assign_polys.py`.

In this case, the column is **`'CL_ID'`**

But keep in mind that this could be different in a different dataset. Look for something that looks like it means "centerline id" or "polygon id".

We can't do anything with rows that do not have a polygon id, so the next step will be to drop any rows with a missing polygon id

In [89]:
# Make a variable for the polygon id column name
# Change this to the name you identified in the list above
poly_id_col = "Poly_ID_01"

print(len(df_data))
df_data = df_data.dropna(subset=[poly_id_col])
print(len(df_data))

9432
9415


We can see that the length updated if there were any rows missing a polygon id. Now we are ready to compute statistics.

# 3. Compute statistics

In [90]:
# First group the data by poly_id_col
df_grouped = df_data.groupby(poly_id_col)

In [91]:
# Now lets define the statistics we want to compute in a list that we can pass to the pandas aggregation function
# For more information on what can go into this list check out: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html
stats = [
    "min",
    "max",
    "mean",
    "std",
    "median"
]

# Compute the statistics defined above for each polygon
df_stats = df_grouped.agg(stats)

In [92]:
df_stats

WS_fDOM_QSU                                  WS_fCHLA_ug/L  \
                   min        max       mean     median           min   
Poly_ID_01                                                              
658.0        20.307835  22.343563  20.672524  20.458392      1.578609   
659.0        20.372952  22.375110  20.688638  20.464035      1.579617   
660.0        20.398193  22.360771  21.147275  20.458750      1.591391   
661.0        20.240320  22.350891  21.361242  22.309642      1.593953   
662.0        19.870817  22.303481  21.107422  21.188231      1.589736   
...                ...        ...        ...        ...           ...   
2522.0       20.521118  22.519637  20.766443  20.580666      1.582137   
2523.0       20.436851  22.609304  20.806354  20.582530      1.581960   
2524.0       20.528359  22.613932  21.384051  20.641275      1.578737   
2525.0       20.299532  22.425153  20.674250  20.389524      1.580410   
2934.0       20.614642  20.617716  20.615666  20.614642      1.715820   

                                         SUNANO3uM          ... EXOTurbFNU  \
                 max      mean    median       min     max  ...       mean   
Poly_ID_01                                                  ...              
658.0       1.762222  1.715953  1.730759    14.940  15.960  ...   3.066652   
659.0       1.733389  1.697334  1.709421    15.155  16.030  ...   3.155637   
660.0       1.702779  1.661541  1.697608    15.275  16.035  ...   2.931579   
661.0       1.705511  1.644808  1.604446    15.290  16.040  ...   2.766552   
662.0       1.716463  1.651032  1.649854    15.360  16.025  ...   2.725625   
...              ...       ...       ...       ...     ...  ...        ...   
2522.0      1.717050  1.682176  1.690763    14.400  15.715  ...   3.039825   
2523.0      1.724106  1.699267  1.718231    14.570  15.750  ...   3.042385   
2524.0      1.718795  1.659827  1.709357    15.040  15.765  ...   2.991216   
2525.0      1.762940  1.725445  1.749507    14.940  15.800  ...   3.036733   
2934.0      1.722513  1.719755  1.720933    15.690  15.710  ...   3.000000   

                   EXOfDOM_QSU                                  EXOfChla_ug/L  \
            median         min        max       mean     median           min   
Poly_ID_01                                                                      
658.0       3.0950   18.987562  19.216638  19.122831  19.126949           0.0   
659.0       3.1950   18.880557  19.253161  19.092756  19.079351           NaN   
660.0       3.0375   18.880557  19.074040  18.964274  18.955163           NaN   
661.0       2.7050   18.839319  18.982532  18.938729  18.945403           NaN   
662.0       2.7350   18.407220  18.874238  18.633186  18.684796           NaN   
...            ...         ...        ...        ...        ...           ...   
2522.0      3.0175   19.008111  19.384770  19.186363  19.196693           0.0   
2523.0      3.0550   19.008299  19.420661  19.185687  19.169169           NaN   
2524.0      3.1100   19.247676  19.521828  19.389040  19.341477           NaN   
2525.0      3.0900   19.007304  19.519488  19.133666  19.106439           0.0   
2934.0      3.0000   19.186460  19.224089  19.205267  19.205251           NaN   

                                          
                 max      mean    median  
Poly_ID_01                                
658.0       0.000000  0.000000  0.000000  
659.0            NaN       NaN       NaN  
660.0            NaN       NaN       NaN  
661.0            NaN       NaN       NaN  
662.0            NaN       NaN       NaN  
...              ...       ...       ...  
2522.0      0.000000  0.000000  0.000000  
2523.0           NaN       NaN       NaN  
2524.0           NaN       NaN       NaN  
2525.0      0.018983  0.009491  0.009491  
2934.0           NaN       NaN       NaN  

[296 rows x 68 columns]

# 4. Save the result

In [93]:
# Output filepath
# Make this whatever you'd like, I'm just appending "_stats" to the original filename

out_dir = "C:\\Users\\tbergamaschi\\Data\\RMP\\ToeDrainFlowPulse2018\\Statistics"
out_fname = data_path.name.split(".")[0] + "_stats.csv"
out_path = Path(out_dir, out_fname)

# Flatten the hierarchical columns
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

# Write the csv
df_stats.to_csv(out_path)

And thats a wrap!